**Abstract**: *On average, Algeria experiences 1,636 forest fires every year. These forest fires burn 35,024 ha of forest lands annually. Scientists have also determined that forest fires explain 90% of Algerian forest land degradation. The majority of these fires (80%) are of unknown origin, making them difficult to anticipate. Our goal was to create a model that will accurately predict the chance of a forest fire occurring based on climate data. A major stakeholder for this project is the Algerian government. Reducing forest fires has been a major governmental initiative, especially since devastating fires in early 2022. Our model could help inform more efficient wildfire management programs. Additional stakeholders include the Algerian people and numerous wildlife organizations and researchers. We developed a logistic regression model that uses interactions between climate and forest fire index predictors that does not fail to predict a forest fire when one occurs (100% recall). We recommend stakeholders optimize data collection of the predictors used in our model and employ a similar model to inform wildfire preparation efforts and processes.*

## Length of the report {-}
The length of the report must be no more than 15 pages, when printed as PDF. However, there is no requirement on the minimum number of pages.

You may put additional stuff as Appendix. You may refer to the Appendix in the main report to support your arguments. However, your appendix is unlikely to be checked while grading, unless the grader deems it necessary. The appendix, references, and information about GitHub and individual contribution will not be included in the page count, and there is no limit on the length of the appendix.

**Delete this section from the report, when using this template.** 

## Code should be put separately in the code template {-}
Your report should be in a research-paper like style. If there is something that can only be explained by showing the code, then you may put it, otherwise do not put the code in the report. We will check your code in the code template. 

**Delete this section from the report, when using this template.** 

## Background / Motivation

What motivated you to work on this problem?

Mention any background about the problem, if it is required to understand your analysis later on.

## Problem statement 

Describe your problem statement. Articulate your objectives using absolutely no jargon. Interpret the problem as inference and/or prediction.

## Data sources
What data did you use? Provide details about your data. Include links to data if you are using open-access data.

## Stakeholders
Who cares? If you are successful, what difference will it make to them?

## Data quality check / cleaning / preparation 

In a tabular form, show the distribution of values of each variable used in the analysis - for both categorical and continuous variables. Distribution of a categorical variable must include the number of missing values, the number of unique values, the frequency of all its levels. If a categorical variable has too many levels, you may just include the counts of the top 3-5 levels. 

If the tables in this section take too much space, you may put them in the appendix, and just mention any useful insights you obtained from the data quality check that helped you develop the model or helped you realize the necessary data cleaning / preparation.

Were there any potentially incorrect values of variables that required cleaning? If yes, how did you clean them? 

Did you do any data wrangling or data preparation before the data was ready to use for model development? Did you create any new predictors from exisiting predictors? For example, if you have number of transactions and spend in a credit card dataset, you may create spend per transaction for predicting if a customer pays their credit card bill. Mention the steps at a broad level, you may put minor details in the appendix. Only mention the steps that ended up being useful towards developing your final model(s).

## Exploratory data analysis

Put the relevant EDA here (visualizations, tables, etc.) that helped you figure out useful predictors for developing the model(s). Only put the EDA that ended up being useful towards developing your final model(s). 

List the insights (as bullet points) you got from EDA that ended up being useful towards developing your final model. 

Again, if there are too many plots / tables, you may put them into appendix, and just mention the insights you got from them.

## Approach

What kind of a model (linear / logistic / other) did you use? What performance metric(s) did you optimize and why?

Is there anything unorthodox / new in your approach? 

What problems did you anticipate? What problems did you encounter? Did the very first model you tried work? 

Did your problem already have solution(s) (posted on Kaggle or elsewhere). If yes, then how did you build upon those solutions, what did you do differently? Is your model better as compared to those solutions in terms of prediction / inference?

**Important: Mention any code repositories (with citations) or other sources that you used, and specifically what changes you made to them for your project.**

We used a logistic approach as it was a classification problem. We chose to optimize for recall rather than precision or accuracy. This was due to the context of our problem. In our opinion, it is better for our model to cause a false alarm (i.e. predict a fire when none occurs) than fail to predict a fire when one actually occurs. In the latter case, people would be completely blindsided by the fire and the damage done would be much worse. When optimizing our model using forward and backward stepwise selection, we picked the model with the lowest AIC as opposed to BIC. This is because we wanted our model to be less conservative for the reasons mentioned above.

There is nothing uniquely unorthodox or new in our approach. One major problem we anticipated was collinearity between our predictors. As mentioned above, a lot of our predictors (especially the FWI indexes) are derived from the same "base" climate variables (temperature, rain, humidity, etc...). As such, we anticpated and experienced severe collinearity between our predictors. Our initial thought was to perform a VIF test (seen below) and select variables where the VIF test was below 5. This ended up changing for reasons that will be explained in the "Developing the Model" section.

Our code did already have some solutions on GitHub, but they were mostly using different model building than methods than we learned in class. As most used a combination of different methods, they ended up being slightly better than our model. However, our model is the only model that I found that can hit 100% Recall consistently.

## Developing the model

Explain the steps taken to develop and improve the base model - informative visualizations / addressing modeling assumption violations / variable transformation / interactions / outlier treatment / influential points treatment / addressing over-fitting / addressing multicollinearity / variable selection - stepwise regression, lasso, ridge regression). 

Did you succeed in achieving your goal, or did you fail? Why?

**Put the final model equation**.

**Important: This section should be rigorous and thorough. Present detailed information about decision you made, why you made them, and any evidence/experimentation to back them up.**

Our base model initially did not consist of all the predictors. As mentioned earlier, many of our variables were strongly correlated. The logit model, which uses a Hessian Matrix for optimization by default, would throw an error. As such, our base model consisted of the following variables: Temperature, RH, Ws, Rain, and FFMC. These variables had a VIF score of less than 5. It was quite accurate when it came to the training data and fairly accurate when it came to the test data. You can see the confusion matrix shown below: 

In [5]:
# confusion matrix for VIF model

Eventually, we were able to get all of the variables work in a basic logit model by changing the optimization method from the Hessian Matrix to the Broyden–Fletcher–Goldfarb–Shanno algorithm. This consisted of nothing more than adding method = 'bgfs' to the logit model. The results were not significantly different from the VIF tested model. 

What was most concerning was the 5 false negatives. Even though the precision and accuracy were both fairly high, our goal was to maximize recall at whatever cost. First, we checked to see if any transformations were necessary. We binned each of the variables and tried to identify any potential non-linear trends. There didn't seem to be any, and when we tested quadratic and cubic terms it hurt the model significantly. The plots of the binned variables vs the percentages of fires can be seen in the code notebook. 

Our main technique was performing forward stepwise selection to find the best possible subsets of predictors and interactions. First, we performed a basic forward stepwise selection. As mentioned above, we chose the model with the lowest AIC because we wanted the model to be more aggressive. That returned a subset of six predictors, which can be seen in the code notebook. It excelled on the training data (100% recall) and did very well on the test data (97.4%) recall. Still, we felt like we could do better

Next, we performed forward stepwise selection to find the best possible subset of all potential degree two interaction terms. That returned a model of 35 predictors. This can also be seen in the code notebook. This performed better on the training data (100% recall) but slightly worse on the test data (92.3% recall). We felt that this was indicative of overfitting. 

Our third and our most successful approach was performing forward stepwise selection on all possible interactions derived from the subset of predictors returned by our basic forward stepwise selection (those 6 predictors mentioned two paragraphs ago). This was the most successful model and returned a model with 13 predictors. The best subset plots can be seen below. 

In [6]:
# best subset plots

This model performed very well on the train data and pretty well on the test data. The confusion matrix can be shown below. 

In [7]:
# confusion matrix for best model

At this point, we felt our work was sufficient. Our model had hit 100% recall which was the major goal. Even though accuracy and precision were worse than the base model, we optimized the most important metric. We did end up testing a few more ideas, including degree three, four, and five interactions but those ended up being significantly worse. The results of the degree four interaction stepwise selection can be seen in the code notebook. I believe we achieved our goal

Our final model equation is: <br>
Classes = FFMC + FWI*RH + BUI*Ws + FWI*Rain + RH + FFMC*RH + FFMC*FWI + FWI + FFMC*Ws + BUI + RH*Ws + FFMC*BUI + RH*BUI

## Limitations of the model with regard to inference / prediction

If it is inference, will the inference hold for a certain period of time, for a certain subset of population, and / or for certain conditions.

If it is prediction, then will it be possible / convenient / expensive for the stakeholders to collect the data relating to the predictors in the model. Using your model, how soon will the stakeholder be able to predict the outcome before the outcome occurs. For example, if the model predicts the number of bikes people will rent in Evanston on a certain day, then how many days before that day will your model be able to make the prediction. This will depend on how soon the data that your model uses becomes available. If you are predicting election results, how many days / weeks / months / years before the election can you predict the results. 

When will your model become too obsolete to be useful?

## Other sections *(optional)*

You are welcome to introduce additional sections or subsections, if required, to address any specific aspects of your project in detail. For example, you may briefly discuss potential future work that the research community could focus on to make further progress in the direction of your project's topic.

## Conclusions and Recommendations to stakeholder(s)

What conclusions do you draw based on your model? If it is inference you may draw conclusions based on the coefficients, statistical significance of predictors / interactions, etc. If it is prediction, you may draw conclusions based on prediction accuracy, or other performance metrics.

How do you use those conclusions to come up with meaningful recommendations for stakeholders? The recommendations must be action-items for stakeholders that they can directly implement without any further analysis. Be as precise as possible. The stakeholder(s) are depending on you to come up with practically implementable recommendations, instead of having to think for themselves.

If your recommendations are not practically implementable by stakeholders, how will they help them? Is there some additional data / analysis / domain expertise you need to do to make the recommendations implementable? 

Do the stakeholder(s) need to be aware about some limitations of your model? Is your model only good for one-time use, or is it possible to update your model at a certain frequency (based on recent data) to keep using it in the future? If it can be used in the future, then for how far into the future?

## GitHub and individual contribution {-}

Put the **Github link** for the project repository.

Add details of each team member's contribution in the table below.

<html>
<style>
table, td, th {
  border: 1px solid black;
}

table {
  border-collapse: collapse;
  width: 100%;
}

th {
  text-align: left;
}
    

</style>
<body>

<h2>Individual contribution</h2>

<table style="width:100%">
     <colgroup>
       <col span="1" style="width: 15%;">
       <col span="1" style="width: 20%;">
       <col span="1" style="width: 50%;">
       <col span="1" style="width: 15%;"> 
    </colgroup>
  <tr>
    <th>Team member</th>
    <th>Contributed aspects</th>
    <th>Details</th>
    <th>Number of GitHub commits</th>
  </tr>
  <tr>
    <td>Elton John</td>
    <td>Data cleaning and EDA</td>
    <td>Cleaned data to impute missing values and developed visualizations to identify appropriate variable transformations.</td>
    <td>100</td>
  </tr>
  <tr>
    <td>Xena Valenzuela</td>
    <td>Assumptions and interactions</td>
    <td>Checked and addressed modeling assumptions and identified relevant variable interactions.</td>
    <td>120</td>
  </tr>
    <tr>
    <td>Sankaranarayanan Balasubramanian</td>
    <td>Outlier and influential points treatment</td>
    <td>Identified outliers/influential points and analayzed their effect on the model.</td>
    <td>130</td>    
  </tr>
    <tr>
    <td>Chun-Li</td>
    <td>Variable selection and addressing overfitting</td>
    <td>Performed variable selection on an exhaustive set of predictors to address multicollinearity and overfitting.</td>
    <td>150</td>    
  </tr>
</table>

List the **challenges** you faced when collaborating with the team on GitHub. Are you comfortable using GitHub? 
Do you feel GitHuB made collaboration easier? If not, then why? *(Individual team members can put their opinion separately, if different from the rest of the team)*

## References {-}

List and number all bibliographical references. When referenced in the text, enclose the citation number in square brackets, for example [1].

[1] Authors. The frobnicatable foo filter, 2014. Face and Gesture submission ID 324. Supplied as additional material
fg324.pdf. 3


## Appendix {-}

You may put additional stuff here as Appendix. You may refer to the Appendix in the main report to support your arguments. However, the appendix section is unlikely to be checked while grading, unless the grader deems it necessary.